In [1]:
#pip install lightgbm


In [10]:
#pip install optuna


In [11]:
#pip install -U scikit-learn

https://mariofilho.com/lightgbm-tuning-hiperparametros-optuna/

Ao ajustar seus hiperparâmetros primeiro, você vai espremer cada gota de desempenho do seu modelo com os dados que já tem.  
Depois que você tiver os hiperparâmetros ideais, você passa para a engenharia de features.

A primeira coisa a considerar é o número de árvores que você vai treinar, também conhecido como *num_iterations*.

Quanto mais árvores você tiver, mais estáveis serão suas previsões.

Então, quantas árvores você deve escolher? Depende do caso de uso do seu modelo.

Se o seu modelo precisa entregar resultados com baixa latência (por exemplo: negociação de alta frequência, previsão de cliques em anúncios), você pode limitar o número de árvores para cerca de 200.

No entanto, se o seu modelo é executado uma vez por semana (por exemplo: previsão de vendas) e tem mais tempo para fazer as previsões, você pode considerar usar até 5.000 árvores.
Como regra geral, comece fixando o número de árvores e depois foque em tunar a learning_rate.

Isso controla quanto cada árvore contribui para a previsão final.

Quanto mais árvores você tiver, menor deve ser a taxa de aprendizado.

O intervalo recomendado para a taxa de aprendizado é entre 0,001 e 0,1.
Em seguida, temos o num_leaves.

Isso determina a complexidade de cada árvore em seu modelo.

Você pode pensar nisso como o equivalente ao parâmetro max_depth em outros modelos baseados em árvores.

Refere-se ao número máximo de nós terminais, ou folhas, que cada árvore pode ter.

Em uma árvore de decisão, uma folha representa uma decisão ou um resultado.
Ao aumentar o num_leaves, você permite que a árvore cresça mais complexa, criando um número maior de caminhos de decisão distintos.

Isso pode levar a um modelo mais flexível que pode capturar padrões complexos nos dados.

No entanto, aumentar o número de folhas também pode fazer com que o modelo sofra overfitting dos dados de treinamento, pois terá uma quantidade menor de dados por folha.

Eu gosto de tuná-lo em potências de 2, começando de 2 e indo até 1024.

O hiperparâmetro subsample desempenha um papel no controle da quantidade de dados usados para construir cada árvore em seu modelo.

É uma fração que varia de 0 a 1, representando a proporção do conjunto de dados a ser selecionada aleatoriamente para treinar cada árvore.

Ao usar apenas um subconjunto dos dados para cada árvore, o modelo pode se beneficiar da diversidade e reduzir a correlação entre as árvores, o que pode ajudar a combater o overfitting.

Lembre-se de definir bagging_freq como um valor positivo ou o LightGBM ignorará o subsample.

bagging_freq é a frequência com que os dados são amostrados.

Defini-lo como 1 significa reamostrar os dados antes da construção de cada árvore, que é o comportamento padrão de outros modelos baseados em árvores.

O intervalo que eu uso para subsample é entre 0,05 e 1.

O hiperparâmetro colsample_bytree é outro aspecto importante a considerar ao tunar seu modelo LightGBM.

Ele determina a proporção de features a serem usadas para cada árvore.

Esse valor varia de 0 a 1, onde um valor de 1 significa que todas as features serão consideradas para cada árvore, e um valor menor indica que apenas um subconjunto de features será escolhido aleatoriamente antes de construir cada árvore.

Este método também é conhecido como Random Subspace.

Os hiperparâmetros subsample e colsample_bytree no LightGBM o tornam semelhante às Random Forests, já que este último amostra linhas (com reposição) e colunas para cada árvore.

Mesmo que esses hiperparâmetros façam o LightGBM e as Random Forests parecerem semelhantes, eles são algoritmos diferentes.

LightGBM é um método de gradient boosting, enquanto Random Forests é um método de bagging, o que significa que eles aprendem com os dados de maneiras diferentes.

Finalmente, o hiperparâmetro min_data_in_leaf define o número mínimo de pontos de dados que devem estar presentes em nós terminais de cada árvore.

Este parâmetro ajuda a controlar a complexidade do modelo e previne o overfitting.

Pense nisso, se você tem um nó terminal com apenas uma amostra, seu rótulo será o valor desse único ponto de dados.

Se você tem um nó terminal com 30 amostras, seu rótulo será a média desses 30 pontos de dados.

É estatisticamente melhor tomar decisões com base em mais pontos de dados.

Isso não significa que você deva definir min_data_in_leaf com um valor alto, pois isso tornará seu modelo menos flexível e mais propenso ao underfitting.

Eu gosto de mantê-lo na faixa de 1 a 100.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, delimiter=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
X = data.drop("quality", axis=1)
y = data["quality"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import optuna

C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# Os métodos trial.suggest_* são usados para especificar o espaço de busca para cada hiperparâmetro.
# Por exemplo, a learning_rate é pesquisada dentro de uma escala logarítmica de 1e-3 a 0,1, e num_leaves é pesquisado dentro de um intervalo inteiro de 2 a 1024.
#A escala logarítmica é usada para a taxa de aprendizado porque tentará mais valores próximos a 0,001, já que taxas de aprendizado pequenas com um grande número de árvores tendem a ser mais estáveis.
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 1000,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [28]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2024-08-07 20:18:31,483] A new study created in memory with name: no-name-a127ac6b-5d1d-467f-8261-b440b81ae676
C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-08-07 20:18:32,127] Trial 0 finished with value: 0.630967302384811 and parameters: {'learning_rate': 0.0020834529080057495, 'num_leaves': 205, 'subsample': 0.7345167174218623, 'colsample_bytree': 0.23518091333262314, 'min_data_in_leaf': 34}. Best is trial 0 with value: 0.630967302384811.
C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 a

In [29]:
print('Melhores hiperparâmetros: ', study.best_params)
print('Melhor RMSE: ', study.best_value)

Melhores hiperparâmetros:  {'learning_rate': 0.010480177596475996, 'num_leaves': 638, 'subsample': 0.23125773596105187, 'colsample_bytree': 0.6529908719832254, 'min_data_in_leaf': 2}
Melhor RMSE:  0.5232999342981532
